In [1]:
import os
import cv2
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms, models
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder

In [9]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("CUDA(GPU)를 사용합니다.")
else:
    device = torch.device("cpu")
    print("CUDA(GPU)를 사용할 수 없으므로, CPU를 사용합니다.")

CUDA(GPU)를 사용할 수 없으므로, CPU를 사용합니다.


In [36]:
# 데이터 불러오기
df = pd.read_csv('styles.csv', on_bad_lines='skip')

In [39]:
# 필요한 컬럼 추출
df = df[['id', 'subCategory']]

# 카테고리 변경
df.loc[df['subCategory'].isin(['Sandal', 'Flip Flops']), 'subCategory'] = 'Shoes'
df.loc[df['subCategory'].isin(['Dress']), 'subCategory'] = 'Topwear'

# Label 데이터프레임 생성
label_df = df[df['subCategory'].isin(['Topwear', 'Shoes', 'Bags','Bottomwear', 'Watches', 'Innerwear', 'Eyewear', 'Wallets', 'Belts'])]

In [40]:
label_df

,id,subCategory
0,15970,Topwear
1,39386,Bottomwear
2,59263,Watches
3,21379,Bottomwear
4,53759,Topwear
...,...,...
44418,42234,Topwear
44419,17036,Shoes
44420,6461,Shoes
44421,18842,Topwear


In [44]:
# 파일 크기 조정
target_size = (56, 56)  # 원하는 크기로 변경

# 조정한 파일 리스트화
resized_images = []

for filename in os.listdir('images'):
    filepath = os.path.join('images', filename)
    img = cv2.imread(filepath)
    resized_img = cv2.resize(img, target_size)
    resized_images.append(resized_img)

In [15]:
# 파일 불러오기
dir = './resized_images/'  # 경로 지정
train_images = []

for i in range(len(df[:10000])):
    try:
        file = str(df.id[i]) + '.jpg'
        img = cv2.imread(dir + file)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        train_images.append(img)
    except:
        pass

train_images = np.array(train_images)
train_images = train_images / 255.0  # 정규화
train_images.shape

(9993, 224, 224, 3)

In [ ]:
encoder = LabelEncoder()

labels = encoder.fit_transform(label_df.subCategory.unique())

labels